<a href="https://colab.research.google.com/github/tanya0131/Deepfake-video-detection/blob/main/deepfake_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!pip install efficientnet_pytorch
!pip install opencv-python-headless

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 104.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 74.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 48.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 89.3 MB/s eta 0:00:00
  Created wheel for efficientnet_pytorch: filename=efficientnet_pytorch-0.7.1-py3-none-any.whl size=16426 sha256=d926e085191a994f8197c5e1cfe10d1af9e52b624e894de0

In [3]:
import os
import cv2
import json
import numpy as np
from tqdm import tqdm
from glob import glob
from PIL import Image
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import Dataset, DataLoader
from efficientnet_pytorch import EfficientNet

In [4]:
DATA_DIR = "/content/drive/MyDrive/dataset"
TRAIN_VIDEO_DIR = os.path.join(DATA_DIR, "train_videos")
TEST_VIDEO_DIR = os.path.join(DATA_DIR, "test_video")
FLOW_IMG_DIR = os.path.join(DATA_DIR, "flow_images")
os.makedirs(FLOW_IMG_DIR, exist_ok=True)


In [5]:
metadata_path = os.path.join(TRAIN_VIDEO_DIR, 'metadata.json')
with open(metadata_path, 'r') as f:
    metadata = json.load(f)

In [6]:
def compute_and_save_optical_flow(video_path, save_dir):
    cap = cv2.VideoCapture(video_path)
    ret, prev_frame = cap.read()
    if not ret:
        return
    prev_gray = cv2.cvtColor(prev_frame, cv2.COLOR_BGR2GRAY)
    count = 0

    while True:
        ret, frame = cap.read()
        if not ret:
            break
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        flow = cv2.calcOpticalFlowFarneback(prev_gray, gray, None, 0.5, 3, 15, 3, 5, 1.2, 0)

        mag, ang = cv2.cartToPolar(flow[..., 0], flow[..., 1])
        hsv = np.zeros_like(frame)
        hsv[..., 1] = 255
        hsv[..., 0] = ang * 180 / np.pi / 2
        hsv[..., 2] = cv2.normalize(mag, None, 0, 255, cv2.NORM_MINMAX)
        rgb_flow = cv2.cvtColor(hsv, cv2.COLOR_HSV2BGR)

        out_path = os.path.join(save_dir, f"flow_{count:03d}.jpg")
        cv2.imwrite(out_path, rgb_flow)
        count += 1
        prev_gray = gray
    cap.release()

In [7]:
train_videos = glob(os.path.join(TRAIN_VIDEO_DIR, '*.mp4'))
for video in tqdm(train_videos[:10]):  # limit to 10 videos for Colab resource management
    vid_name = os.path.basename(video)
    if vid_name not in metadata:
        continue
    label = metadata[vid_name]['label']
    out_dir = os.path.join(FLOW_IMG_DIR, label, os.path.splitext(vid_name)[0])
    os.makedirs(out_dir, exist_ok=True)
    compute_and_save_optical_flow(video, out_dir)

100%|██████████| 10/10 [1:09:01<00:00, 414.17s/it]


In [7]:
class FlowDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.samples = []
        self.transform = transform
        for label in ['REAL', 'FAKE']:
            class_dir = os.path.join(root_dir, label)
            for folder in os.listdir(class_dir):
                img_dir = os.path.join(class_dir, folder)
                imgs = glob(os.path.join(img_dir, '*.jpg'))
                for img in imgs:
                    self.samples.append((img, 0 if label == 'REAL' else 1))

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        img_path, label = self.samples[idx]
        image = Image.open(img_path).convert('RGB')
        if self.transform:
            image = self.transform(image)
        return image, label

In [8]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
])

dataset = FlowDataset(FLOW_IMG_DIR, transform=transform)
loader = DataLoader(dataset, batch_size=16, shuffle=True)

In [9]:
model = EfficientNet.from_pretrained('efficientnet-b0', num_classes=2)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-4)

Downloading: "https://github.com/lukemelas/EfficientNet-PyTorch/releases/download/1.0/efficientnet-b0-355c32eb.pth" to /root/.cache/torch/hub/checkpoints/efficientnet-b0-355c32eb.pth
100%|██████████| 20.4M/20.4M [00:00<00:00, 231MB/s]

Loaded pretrained weights for efficientnet-b0


In [10]:
model.train()
for epoch in range(1):
    running_loss = 0.0
    correct = 0
    total = 0
    for inputs, labels in tqdm(loader):
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print(f"Epoch {epoch+1}, Loss: {running_loss:.4f}, Accuracy: {correct/total:.4f}")


100%|██████████| 187/187 [29:13<00:00,  9.38s/it]

Epoch 1, Loss: 30.0411, Accuracy: 0.9605


In [11]:
TEST_FLOW_IMG_DIR = os.path.join(DATA_DIR, "test_flow_images")
os.makedirs(TEST_FLOW_IMG_DIR, exist_ok=True)

test_videos = glob(os.path.join(TEST_VIDEO_DIR, '*.mp4'))

for video in tqdm(test_videos[:10]):  # limit to a few for demo
    vid_name = os.path.splitext(os.path.basename(video))[0]
    out_dir = os.path.join(TEST_FLOW_IMG_DIR, vid_name)
    os.makedirs(out_dir, exist_ok=True)
    compute_and_save_optical_flow(video, out_dir)

100%|██████████| 10/10 [54:59<00:00, 329.99s/it]


In [12]:
class TestFlowDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.samples = []
        self.transform = transform
        self.video_to_imgs = {}
        for folder in os.listdir(root_dir):
            img_dir = os.path.join(root_dir, folder)
            imgs = sorted(glob(os.path.join(img_dir, '*.jpg')))
            for img in imgs:
                self.samples.append((img, folder))
            self.video_to_imgs[folder] = imgs

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        img_path, video_id = self.samples[idx]
        image = Image.open(img_path).convert('RGB')
        if self.transform:
            image = self.transform(image)
        return image, video_id

In [13]:
test_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
])

test_dataset = TestFlowDataset(TEST_FLOW_IMG_DIR, transform=test_transform)
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False)

model.eval()
video_preds = {}
with torch.no_grad():
    for inputs, video_ids in tqdm(test_loader):
        inputs = inputs.to(device)
        outputs = model(inputs)
        probs = torch.softmax(outputs, dim=1)[:, 1]
        for i, vid in enumerate(video_ids):
            if vid not in video_preds:
                video_preds[vid] = []
            video_preds[vid].append(probs[i].item())

final_preds = {}
for vid, probs in video_preds.items():
    avg_prob = np.mean(probs)
    final_label = 'fake' if avg_prob > 0.5 else 'real'
    final_preds[vid] = final_label


100%|██████████| 187/187 [05:26<00:00,  1.75s/it]


In [14]:
print(final_preds)

{'yljecirelf': 'real', 'zfrrixsimm': 'fake', 'yllztsrwjw': 'real', 'zcxcmneefk': 'real', 'zfobicuigx': 'fake', 'ylxwcwhjjd': 'real', 'zgjosltkie': 'real', 'zgbhzkditd': 'real', 'zmxeiipnqb': 'real', 'ziipxxchai': 'real'}
